In [2]:
%pip install langchain sentence_transformers lancedb unstructured

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/89/b2/3b74b85356662637bfe3efbc6462ccb28227215fcf8e07b5e9a830f5c661/langchain-0.0.275-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for lancedb from https://files.pythonhosted.org/packages/0b/59/8800508af2c0afe0269278432ed5a7a8b03bebc75826ac211ee4053bf7a1/lancedb-0.2.2-py3-none-any.whl.metadata
  Obtaining dependency information for unstructured from https://files.pythonhosted.org/packages/e3/97/478e5f01e8922acc140ee35adde4c5e6861f7a693fe57daafe53947e6602/unstructured-0.10.8-py3-none-any.whl.metadata
  Obtaining dependency information for dataclasses-json<0.6.0,>=0.5.7 from https://files.pythonhosted.org/packages/97/5f/e7cc90f36152810cab08b6c9c1125e8bcb9d76f8b3018d101b5f877b3

In [3]:
import langchain
import langchain.document_loaders

source_dir = "/home/jupyter/mnt/s3/mclass/text"

loader = langchain.document_loaders.DirectoryLoader(source_dir,glob="*.txt",show_progress=True,recursive=True)
splitter = langchain.text_splitter.RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=50)
fragments = splitter.create_documents([ x.page_content for x in loader.load()])
len(fragments)

  0%|          | 0/1 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 1/1 [00:07<00:00,  7.62s/it]


214

In [11]:
embeddings = langchain.embeddings.HuggingFaceEmbeddings(model_name="distiluse-base-multilingual-cased-v1")
sample_vec = embeddings.embed_query("Hello, world!")
len(sample_vec)

512

In [12]:
from langchain.vectorstores import LanceDB
import lancedb

db_dir = "../store"

db = lancedb.connect(db_dir)
table = db.create_table(
    "vector_index",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)

In [13]:
db = LanceDB.from_documents(fragments, embeddings, connection=table)

In [17]:
q="Что такое гибридные облака?"

res = db.similarity_search(q)
for x in res:
    print(x.page_content)

облачные технологии и после того когда Я прослушала конференция записала около 7 вопросов И придя на чехлом я спросила Вопрос уже как мы идем в облака Не нужны ли нам облачные технологии а как мы идем в облака в каких проектах мы можем использовать Гибридное частное облако надо какие сервисы есть сейчас В теха Решениях которые на рынке на рынке не так много решений и После этого когда Родились эти вопросы как мы идем в облака совершенно по другому заработали технологические команды и Начали предлагать и
чтобы кибербезопасность Не упала в обморок и не написала отрицательное заключение мы использовали гибридное облако
технологические команды и Начали предлагать и отвечать где какие сервисы и какие возможности сдаст нам облака 2 разных совершенно вопроса нужны ли нам облака нет не нужны А вопрос как мы будем использовать эти сервисы дальше была обыкновенная табличка в несколько компаний попали А в шорт лист как сделать выбор и абсолют банк сделал выбор в пользу яндекс облака Я вам расскаж

In [20]:
retriever = db.as_retriever(
    search_kwargs={"k": 5}
)
res = retriever.get_relevant_documents(q)

In [33]:
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
import requests

class YandexLLM(langchain.llms.base.LLM):
    iam_token: str
    folder_id: str
    max_tokens : int = 1500
    temperature : float = 1
    instruction_text : str = None

    @property
    def _llm_type(self) -> str:
        return "yagpt"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        req = {
          "model": "general",
          "instruction_text": self.instruction_text,
          "request_text": prompt,
          "generation_options": {
            "max_tokens": self.max_tokens,
            "temperature": self.temperature
          }
        }
        res = requests.post("https://llm.api.cloud.yandex.net/llm/v1alpha/instruct",
          headers=
            { "Authorization" : f"Bearer {self.iam_token}",
              "x-folder-id" : self.folder_id }, json=req).json()
        return res['result']['alternatives'][0]['text']

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"max_tokens": self.max_tokens, "temperature" : self.temperature }

In [34]:
iam = "t1.9euelZqZlZqUyY_HiouXk8jMnI2di-3rnpWalpOXl5iQzJqTyZeVzJaRl5vl8_cAHUxY-e9qTyJY_N3z90BLSVj572pPIlj8zef1656VmonOm4zHjMyVlc-Mj5ibjpGS7_zF656VmonOm4zHjMyVlc-Mj5ibjpGS.Xw3IuhSfz1C1IPfjsFChbbQ429A7X33S0Ip5y6X0le1ceV2uL0r2xTR2fb8MI1ZveYZQTe8Lz9rzcGt1Imx2Cw"

instructions = """
Представь себе, что ты работник техподдержки яндекс-облака. Твоя задача - подробно отвечать на все вопросы."""

llm = YandexLLM(iam_token=iam, folder_id="b1gbicod0scglhd49qs0",
                instruction_text = instructions)

In [35]:
llm("Как получить iam-токен?")

'Добрый день! \n\nЕсли вам нужен `IAM-токен` для доступа к API Яндекс.Облака, вы можете запросить его в любом из наших продуктов и приложений. Список доступных продуктов и API доступен на странице https://tech.yandex.ru/cloud. В зависимости от продукта или API, вам понадобится `API-ключ` или `IAM-роли`, которые вы также можете получить через API. Если вы хотите узнать больше о продуктах или API Яндекс.Облако, пожалуйста, посмотрите на страницу https://www.ybxmission.com/.\n\nВы также можете подать заявку на это на [очередь операционных задач](https://cloud.yandex.ru/).'

In [37]:
# We prepare and run a custom Stuff chain with reordered docs as context.

# Override prompts
document_prompt = langchain.prompts.PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)
document_variable_name = "context"
stuff_prompt_override = """Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста.
Текст:
-----
{context}
-----
Вопрос:
{query}"""
prompt = langchain.prompts.PromptTemplate(
    template=stuff_prompt_override, input_variables=["context", "query"]
)

# Instantiate the chain
llm_chain = langchain.chains.LLMChain(llm=llm, prompt=prompt)
chain = langchain.chains.StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)
chain.run(input_documents=res, query=q)

'Гибридные облака — это комбинация нескольких типов облаков, которые объединяются для улучшения безопасности и эффективности. То есть, в гибридном облаке используются как публичные, так и частные сценарии. Публичным облаком называют коммерческие облака, таких как Google Cloud, Amazon Web Services.'

In [38]:
from langchain.document_transformers import LongContextReorder
reorderer = LongContextReorder()

def answer(query,reorder=True):
  results = retriever.get_relevant_documents(query)
  if reorder:
    results = reorderer.transform_documents(results)
  return chain.run(input_documents=results, query=query)

In [39]:
answer("Зачем нужно облако?")

'Вкратце, облако больше, чем резервный центр обработки данных, серверы, процессоры и хранилища. Возможности облака основаны на технологической платформе и библиотеках контейнерного развертывания и можно использовать для автоматизации разработки пользовательских приложений, тестирования их, организации тотальной изоляции изменений и эксплуатации.'